In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence

# Load the dataset
pet_shop_data = pd.read_csv('1654342Pet shop.csv')



In [2]:
# Convert categorical variables into dummy variables, drop first to avoid dummy variable trap
pet_shop_data = pd.get_dummies(pet_shop_data, columns=['product_category', 'quality'], drop_first=True)

# Dropping the unnamed index column if present
if 'Unnamed: 0' in pet_shop_data.columns:
    pet_shop_data.drop(columns='Unnamed: 0', inplace=True)

# Initially, handle missing values by simply dropping rows with missing values to prepare for Cook's D analysis
pet_shop_data_clean = pet_shop_data.dropna()

# Define independent variables (X) and the dependent variable (y)
X = pet_shop_data_clean.drop('products_sold', axis=1)
y = pet_shop_data_clean['products_sold']

# Add a constant to the independent variables set for statsmodels
X_with_constant = sm.add_constant(X)

# Create and fit the model using statsmodels OLS
model = sm.OLS(y, X_with_constant).fit()

# Calculate Cook's D to identify outliers
influence = model.get_influence()
cooks_d = influence.cooks_distance[0]
outlier_threshold = 4 / len(X)
outliers_indices = np.where(cooks_d > outlier_threshold)[0]

# Mark identified outliers as NaN in the original dataset before dropping missing values
for index in outliers_indices:
    if index in pet_shop_data_clean.index:
        pet_shop_data.loc[index, 'products_sold'] = np.nan

# After marking outliers as NaN, you can decide on how to handle these missing values, 
# either by imputation or further analysis.
# Note: Further steps for imputation or analysis of missing values will depend on specific methodologies you prefer.

# Display information about the modified dataset
print(pet_shop_data.info())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).